In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="mobilenet_distilled_v2.tflite")
interpreter.allocate_tensors()

# Get input details (shape, dtype)
input_details = interpreter.get_input_details()
input_shape = input_details[0]['shape']  # e.g. [1, height, width, 3]
input_dtype = input_details[0]['dtype']

print(f"Model expects input shape: {input_shape} and dtype: {input_dtype}")

def load_and_preprocess_image(image_path, input_shape, input_dtype):
    # Open image and convert to RGB
    img = Image.open(image_path).convert('RGB')

    # Resize to model input size (width, height)
    width, height = input_shape[3], input_shape[2]  # note swapped here!
    img = img.resize((width, height))

    # Convert image to numpy array (channels last)
    input_data = np.array(img)

    # Normalize if float input expected
    if np.issubdtype(input_dtype, np.floating):
        input_data = input_data.astype(np.float32) / 255.0
    else:
        input_data = input_data.astype(input_dtype)

    # Add batch dimension
    input_data = np.expand_dims(input_data, axis=0)  # shape: [1, height, width, 3]

    # Transpose to channels first: [1, 3, height, width]
    input_data = input_data.transpose(0, 3, 1, 2)

    return input_data


# # Example image path
# image_path = "dfsjh.jpeg"

# # Preprocess image based on model input
# input_data = load_and_preprocess_image(image_path, input_shape, input_dtype)

# # Set tensor and run inference
# interpreter.set_tensor(input_details[0]['index'], input_data)
# interpreter.invoke()

# # Get output
# output_details = interpreter.get_output_details()
# output_data = interpreter.get_tensor(output_details[0]['index'])

# print("Model output:", output_data)
# print("Predicted class:", np.argmax(output_data))


In [22]:
import numpy as np
import tensorflow as tf
from PIL import Image
import pandas as pd
from pathlib import Path

# Load your TFLite model globally once
interpreter = tf.lite.Interpreter(model_path="mobilenet_distilled_v2.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']  # e.g. [1, 3, 256, 256]
input_dtype = input_details[0]['dtype']

def preprocess_image(image_path, input_shape, input_dtype):
    img = Image.open(image_path).convert('RGB')
    width, height = input_shape[3], input_shape[2]  # channels first model
    img = img.resize((width, height))
    input_data = np.array(img)

    if np.issubdtype(input_dtype, np.floating):
        input_data = input_data.astype(np.float32) / 255.0
    else:
        input_data = input_data.astype(input_dtype)

    input_data = np.expand_dims(input_data, axis=0)      # [1, H, W, C]
    input_data = input_data.transpose(0, 3, 1, 2)        # [1, C, H, W]
    return input_data

In [23]:
def get_predictions(image_path):
    input_data = preprocess_image(image_path, input_shape, input_dtype)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    pred_class = np.argmax(output_data)
    probability = output_data[pred_class]

    # Define flag as True if probability > threshold (e.g., 0.5)
    flag = probability > 0.5

    return flag, probability, pred_class

# Define base folder with images
base_path = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\NMMS PHOTOS OF 10TH APRIL")

# Get all images (limit to first 5000)
image_paths = list(base_path.glob('*.jpeg')) + list(base_path.glob('*.jpg')) + list(base_path.glob('*.png'))
# image_paths = image_paths[:5000]

print(f"Found {len(image_paths)} images. Starting prediction..\n")

results = []

for img_path in image_paths:
    try:
        flag, probability, pred_class = get_predictions(img_path)
        results.append({
            'Image': str(img_path),
            'Class': pred_class,
            'Flag': flag,
            'Probability': probability
        })
    except Exception as e:
        print(f"Error with image {img_path.name}: {e}")

df_results = pd.DataFrame(results)

print("\nSample results:")
print(df_results.head())

# Optionally save results to CSV
df_results.to_csv("prediction_results.csv", index=False)

Found 9548 images. Starting prediction..

Error with image 102_1726002003_WC_22012035247758_2025-04-10R1042025111619.jpeg: cannot identify image file 'C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\102_1726002003_WC_22012035247758_2025-04-10R1042025111619.jpeg'
Error with image 1099_1701005060_FP_22012034636038_2025-04-10R1042025101111.jpeg: cannot identify image file 'C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\1099_1701005060_FP_22012034636038_2025-04-10R1042025101111.jpeg'
Error with image 1119_1711003008_LD_22012034946985_2025-04-10R1042025112813.jpeg: cannot identify image file 'C:\\Users\\arpitasharma2\\OneDrive - Deloitte (O365D)\\Desktop\\DIU\\NMMS PHOTOS OF 10TH APRIL\\NMMS PHOTOS OF 10TH APRIL\\1119_1711003008_LD_22012034946985_2025-04-10R1042025112813.jpeg'
Error with image 113_1731002049_RS_22012034654699_2025-04-10R10420

In [ ]:
# import numpy as np
# import tensorflow as tf
# from PIL import Image
# from pathlib import Path
# import pandas as pd

# # Load the TFLite model
# interpreter = tf.lite.Interpreter(model_path="mobilenet_distilled_v2.tflite")
# interpreter.allocate_tensors()

# # Get input and output details
# input_details = interpreter.get_input_details()
# output_details = interpreter.get_output_details()

# # Preprocess image function
# def preprocess_image(image_path, input_shape):
#     img = Image.open(image_path).convert('RGB')
#     img = img.resize((input_shape[3], input_shape[2]))  # width, height
#     img = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0,1]
#     if input_shape[1] == 3:  # If channels is in dim 1 (CHW)
#         img = np.transpose(img, (2, 0, 1))  # HWC -> CHW
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     return img

In [ ]:
# # Define prediction function for one image
# def get_predictions(image_path):
#     input_data = preprocess_image(image_path, input_details[0]['shape'])
#     interpreter.set_tensor(input_details[0]['index'], input_data)
#     interpreter.invoke()
#     output_data = interpreter.get_tensor(output_details[0]['index'])[0]
    
#     predicted_class = np.argmax(output_data)
#     probability = output_data[predicted_class]

#     # For example, you can define flag based on some threshold of probability
#     flag = probability > 0.5

#     return flag, probability, predicted_class

# # Define base folder with images
# base_path = Path(r"C:\Users\arpitasharma2\OneDrive - Deloitte (O365D)\Desktop\DIU\NMMS PHOTOS OF 10TH APRIL\NMMS PHOTOS OF 10TH APRIL")

# # Get all images (limit to first 5000)
# image_paths = list(base_path.glob('*.jpeg')) + list(base_path.glob('*.jpg')) + list(base_path.glob('*.png'))
# image_paths = image_paths[:5000]

# print(f"Found {len(image_paths)} images. Starting prediction..\n")

# # Prediction loop
# results = []

# for img_path in image_paths:
#     try:
#         flag, probability, pred_class = get_predictions(img_path)
#         results.append({
#             'Image': str(img_path),
#             'Class': pred_class,
#             'Flag': flag,
#             'Recaptured_Probability': probability
#         })
#     except Exception as e:
#         print(f"Error with image {img_path.name}: {e}")

# # Create DataFrame and save to CSV
# df_results = pd.DataFrame(results)
# print("\nSample results:")
# print(df_results.head())

In [ ]:
# df_results.to_csv("test.csv", index=False)
# print("\n Results saved to 'test.csv'")